In [25]:
import numpy as np
from numpy import array
from numpy import array
from numpy import asarray
from numpy import zeros
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# set configurations
pd.set_option('display.max_columns', 100)
sns.set_style("white")

# keras imports
import tensorflow as tf
import keras
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten
from keras.layers import GlobalMaxPooling1D, SpatialDropout1D, LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing.text import Tokenizer

# model imports
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer, TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, roc_auc_score
import pickle
import joblib

from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)

In [2]:
model_data = pd.read_csv('../data/scheme1.csv', keep_default_na=False)

In [3]:
model_data.head(3)

,Unnamed: 0,Unnamed: 0.1,title,selftext,author,num_comments,is_suicide,url,selftext_clean,title_clean,author_clean,selftext_length,title_length,megatext_clean,Clustered Labels,New Labels
0,0,0,Our most-broken and least-understood rules is ...,We understand that most people who reply immed...,SQLwitch,133,0,https://www.reddit.com/r/depression/comments/d...,understand people reply immediately op invitat...,broken least understood rule helper may invite...,sql witch,4792,144,sql witch understand people reply immediately ...,1,1
1,1,1,Regular Check-In Post,Welcome to /r/depression's check-in post - a p...,circinia,1644,0,https://www.reddit.com/r/depression/comments/e...,welcome r depression check post place take mom...,regular check post,c irc,650,21,c irc welcome r depression check post place ta...,1,1
2,2,2,I hate it so much when you try and express you...,I've been feeling really depressed and lonely ...,TheNewKiller69,8,0,https://www.reddit.com/r/depression/comments/f...,feeling really depressed lonely lately job ful...,hate much try express feeling parent turn arou...,new killer 69,1866,137,new killer 69 feeling really depressed lonely ...,0,0


In [4]:
model_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1897 entries, 0 to 1896
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Unnamed: 0        1897 non-null   int64 
 1   Unnamed: 0.1      1897 non-null   int64 
 2   title             1897 non-null   object
 3   selftext          1897 non-null   object
 4   author            1897 non-null   object
 5   num_comments      1897 non-null   int64 
 6   is_suicide        1897 non-null   int64 
 7   url               1897 non-null   object
 8   selftext_clean    1897 non-null   object
 9   title_clean       1897 non-null   object
 10  author_clean      1897 non-null   object
 11  selftext_length   1897 non-null   int64 
 12  title_length      1897 non-null   int64 
 13  megatext_clean    1897 non-null   object
 14  Clustered Labels  1897 non-null   int64 
 15  New Labels        1897 non-null   int64 
dtypes: int64(8), object(8)
memory usage: 237.2+ KB


In [5]:
model_data['is_suicide'].mean()

0.5166051660516605

In [45]:
# getting ready for training

X = model_data["selftext_clean"]
y = model_data["is_suicide"]
# train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print('Shape of data tensor:', X_train.shape)

tvec_optimised = TfidfVectorizer(max_df= 0.5, max_features=70, min_df=2, ngram_range=(1, 3),stop_words = 'english')
X_train_tvec = tvec_optimised.fit_transform(X_train).todense()
X_test_tvec = tvec_optimised.transform(X_test).todense()

Shape of data tensor: (1517,)


In [46]:
# X_train_tvec = pad_sequences(X_train_tvec, maxlen=MAX_SEQUENCE_LENGTH)
# print('Shape of data tensor:', X_train_tvec.shape)

In [52]:
model = Sequential()
 
model.add(Dense(units=500, activation='relu', input_dim=len(tvec_optimised.get_feature_names())))
model.add(Dense(units=1, activation='sigmoid'))

In [53]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 500)               35500     
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 501       
Total params: 36,001
Trainable params: 36,001
Non-trainable params: 0
_________________________________________________________________


In [58]:
history = model.fit(X_train_tvec, y_train, batch_size=32, epochs=10, verbose=1, validation_data=(X_test_tvec, y_test))

Train on 1517 samples, validate on 380 samples
Epoch 1/10
1517/1517 [==============================] - 0s 48us/step - loss: 0.4172 - accuracy: 0.8299 - val_loss: 0.6584 - val_accuracy: 0.6553
Epoch 2/10
1517/1517 [==============================] - 0s 61us/step - loss: 0.4013 - accuracy: 0.8477 - val_loss: 0.6469 - val_accuracy: 0.6868
Epoch 3/10
1517/1517 [==============================] - 0s 61us/step - loss: 0.3887 - accuracy: 0.8517 - val_loss: 0.6498 - val_accuracy: 0.6868
Epoch 4/10
1517/1517 [==============================] - 0s 54us/step - loss: 0.3716 - accuracy: 0.8741 - val_loss: 0.6561 - val_accuracy: 0.6816
Epoch 5/10
1517/1517 [==============================] - 0s 55us/step - loss: 0.3547 - accuracy: 0.8761 - val_loss: 0.6632 - val_accuracy: 0.6895
Epoch 6/10
1517/1517 [==============================] - 0s 50us/step - loss: 0.3401 - accuracy: 0.8952 - val_loss: 0.6641 - val_accuracy: 0.6658
Epoch 7/10
1517/1517 [==============================] - 0s 53us/step - loss: 0.3301

In [59]:
scores = model.evaluate(X_test_tvec, y_test, verbose=1)
print("Accuracy:", scores[1])  # Accuracy: 0.875

380/380 [==============================] - 0s 100us/step
Accuracy: 0.6710526347160339


In [75]:
word2idx = {word: idx for idx, word in enumerate(tvec_optimised.get_feature_names())}
tokenize = tvec_optimised.build_tokenizer()
preprocess = tvec_optimised.build_preprocessor()
 
def to_sequence(tokenizer, preprocessor, index, text):
    words = tokenizer(preprocessor(text))
    indexes = [index[word] for word in words if word in index]
    return indexes
 
print(to_sequence(tokenize, preprocess, word2idx, "This is an important test!"))  # [2269, 4453]
X_train_sequences = [to_sequence(tokenize, preprocess, word2idx, x) for x in X_train]
print(X_train_sequences[0])

[]
[15, 40, 57, 64, 19, 55, 41, 58, 40, 64, 64, 47, 23, 67, 64, 63, 49, 61, 61, 36, 55, 21, 47, 23, 56, 64, 64, 44, 0, 47, 36, 12, 32, 21, 15, 9, 65, 9, 30, 6, 2, 14, 57, 36, 60, 53, 18, 9, 30, 65, 54, 36, 40, 2, 30, 47, 12, 30, 55, 51, 21, 47, 23, 56, 58, 58]


In [77]:

# Compute the max lenght of a text
MAX_SEQ_LENGHT = len(max(X_train_sequences, key=len))
print("MAX_SEQ_LENGHT=", MAX_SEQ_LENGHT)
 
from keras.preprocessing.sequence import pad_sequences
N_FEATURES = len(tvec_optimised.get_feature_names())
X_train_sequences = pad_sequences(X_train_sequences, maxlen=MAX_SEQ_LENGHT, value=N_FEATURES)
print(X_train_sequences[0])

MAX_SEQ_LENGHT= 488
[70 70 70 70 70 70 70 70 70 70 70 70 70 70 70 70 70 70 70 70 70 70 70 70
 70 70 70 70 70 70 70 70 70 70 70 70 70 70 70 70 70 70 70 70 70 70 70 70
 70 70 70 70 70 70 70 70 70 70 70 70 70 70 70 70 70 70 70 70 70 70 70 70
 70 70 70 70 70 70 70 70 70 70 70 70 70 70 70 70 70 70 70 70 70 70 70 70
 70 70 70 70 70 70 70 70 70 70 70 70 70 70 70 70 70 70 70 70 70 70 70 70
 70 70 70 70 70 70 70 70 70 70 70 70 70 70 70 70 70 70 70 70 70 70 70 70
 70 70 70 70 70 70 70 70 70 70 70 70 70 70 70 70 70 70 70 70 70 70 70 70
 70 70 70 70 70 70 70 70 70 70 70 70 70 70 70 70 70 70 70 70 70 70 70 70
 70 70 70 70 70 70 70 70 70 70 70 70 70 70 70 70 70 70 70 70 70 70 70 70
 70 70 70 70 70 70 70 70 70 70 70 70 70 70 70 70 70 70 70 70 70 70 70 70
 70 70 70 70 70 70 70 70 70 70 70 70 70 70 70 70 70 70 70 70 70 70 70 70
 70 70 70 70 70 70 70 70 70 70 70 70 70 70 70 70 70 70 70 70 70 70 70 70
 70 70 70 70 70 70 70 70 70 70 70 70 70 70 70 70 70 70 70 70 70 70 70 70
 70 70 70 70 70 70 70 70 70 70 

In [78]:
from keras.models import Sequential
from keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, Embedding
 
model = Sequential()
model.add(Embedding(len(tvec_optimised.get_feature_names()) + 1,
                    64,  # Embedding size
                    input_length=MAX_SEQ_LENGHT))
model.add(Conv1D(64, 5, activation='relu'))
model.add(MaxPooling1D(5))
model.add(Flatten())
model.add(Dense(units=64, activation='relu'))
model.add(Dense(units=1, activation='sigmoid'))

In [79]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_12 (Embedding)     (None, 488, 64)           4544      
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 484, 64)           20544     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 96, 64)            0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 6144)              0         
_________________________________________________________________
dense_13 (Dense)             (None, 64)                393280    
_________________________________________________________________
dense_14 (Dense)             (None, 1)                 65        
Total params: 418,433
Trainable params: 418,433
Non-trainable params: 0
_______________________________________________

In [87]:
X_test_sequences = [to_sequence(tokenize, preprocess, word2idx, x) for x in X_test]
X_test_sequences = pad_sequences(X_test_sequences, maxlen=MAX_SEQ_LENGHT, value=N_FEATURES)

In [88]:
history = model.fit(X_train_sequences, y_train, batch_size=32, epochs=10, verbose=1, validation_data=(X_test_sequences, y_test))

# model.fit(X_train_sequences[:-100], y_train[:-100], 
#           epochs=100, batch_size=64, verbose=1,
#           validation_data=(X_train_sequences[-100:], y_train[-100:]))

Train on 1517 samples, validate on 380 samples
Epoch 1/10
1517/1517 [==============================] - 3s 2ms/step - loss: 0.1548 - accuracy: 0.9242 - val_loss: 2.0098 - val_accuracy: 0.5658
Epoch 2/10
1517/1517 [==============================] - 3s 2ms/step - loss: 0.1484 - accuracy: 0.9262 - val_loss: 1.9223 - val_accuracy: 0.5763
Epoch 3/10
1517/1517 [==============================] - 3s 2ms/step - loss: 0.1463 - accuracy: 0.9255 - val_loss: 1.9247 - val_accuracy: 0.5658
Epoch 4/10
1517/1517 [==============================] - 3s 2ms/step - loss: 0.1515 - accuracy: 0.9222 - val_loss: 1.9728 - val_accuracy: 0.5632
Epoch 5/10
1517/1517 [==============================] - 3s 2ms/step - loss: 0.1483 - accuracy: 0.9229 - val_loss: 1.9642 - val_accuracy: 0.5789
Epoch 6/10
1517/1517 [==============================] - 3s 2ms/step - loss: 0.1474 - accuracy: 0.9249 - val_loss: 1.9712 - val_accuracy: 0.5658
Epoch 7/10
1517/1517 [==============================] - 3s 2ms/step - loss: 0.1465 - accu

In [89]:
scores = model.evaluate(X_test_sequences, y_test, verbose=1)
print("Accuracy:", scores[1])

380/380 [==============================] - 0s 258us/step
Accuracy: 0.5763157606124878


In [91]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding
 
model = Sequential()
model.add(Embedding(len(tvec_optimised.get_feature_names()) + 1,
                    64,  # Embedding size
                    input_length=MAX_SEQ_LENGHT))
model.add(LSTM(64))
model.add(Dense(units=1, activation='sigmoid'))
 
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_13 (Embedding)     (None, 488, 64)           4544      
_________________________________________________________________
lstm_9 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dense_15 (Dense)             (None, 1)                 65        
Total params: 37,633
Trainable params: 37,633
Non-trainable params: 0
_________________________________________________________________
None


In [92]:
history = model.fit(X_train_sequences, y_train, batch_size=32, epochs=10, verbose=1, validation_data=(X_test_sequences, y_test))


Train on 1517 samples, validate on 380 samples
Epoch 1/10
1517/1517 [==============================] - 26s 17ms/step - loss: 0.6785 - accuracy: 0.5570 - val_loss: 0.6790 - val_accuracy: 0.5342
Epoch 2/10
1517/1517 [==============================] - 25s 16ms/step - loss: 0.6419 - accuracy: 0.6388 - val_loss: 0.6508 - val_accuracy: 0.6368
Epoch 3/10
1517/1517 [==============================] - 25s 17ms/step - loss: 0.6124 - accuracy: 0.6724 - val_loss: 0.7001 - val_accuracy: 0.6263
Epoch 4/10
1517/1517 [==============================] - 21s 14ms/step - loss: 0.5938 - accuracy: 0.6856 - val_loss: 0.6500 - val_accuracy: 0.6395
Epoch 5/10
1517/1517 [==============================] - 27s 18ms/step - loss: 0.5786 - accuracy: 0.7053 - val_loss: 0.6441 - val_accuracy: 0.6711
Epoch 6/10
1517/1517 [==============================] - 22s 15ms/step - loss: 0.5653 - accuracy: 0.7086 - val_loss: 0.6348 - val_accuracy: 0.6605
Epoch 7/10
1517/1517 [==============================] - 20s 13ms/step - loss:

In [99]:
import spacy
import numpy as np

import en_core_web_sm
nlp = en_core_web_sm.load()

nlp = spacy.load('en_core_web_md')
 
EMBEDDINGS_LEN = len(nlp.vocab['apple'].vector)
print("EMBEDDINGS_LEN=", EMBEDDINGS_LEN)  # 300
 
embeddings_index = np.zeros((len(tvec_optimised.get_feature_names()) + 1, EMBEDDINGS_LEN))
for word, idx in word2idx.items():
    try:
        embedding = nlp.vocab[word].vector
        embeddings_index[idx] = embedding
    except:
        pass

ModuleNotFoundError: No module named 'en_core_web_sm'

In [ ]:
!pip3 install en_core_web_sm